<span type="title">Spring JDBC and Transaction</span> | <span type="update">2018-10-09</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Spring 对于 JDBC 和事务的支持。在前半部分，主要介绍了 JDBCTemplate 类，这是一个类似于 DBUtils 的 JDBC 工具，可以用来执行增删查改，用来快速的编写 DAO。之后介绍了一个稍微特殊的 NamedParameterJdbcTemplate，此类可改善更新时占位符过多难以分辨的问题，同时支持直接拔取 bean 属性进行数据库更新的操作。在最后介绍了一个叫做 JdbcDaoSupport 的无用的类。总而言之，上半部分主要介绍了 Spring 对于 JDBC 的支持，即快速创建持久层 Data Access Object。</p><p class="card-text">在后半部分，主要介绍了 Spring 对于事务的支持，其实也就是 transactionManager 类，有两种方式使用事务，其一是基于注解，需要创建 manager bean，然后启用事务注解，其二是手动切面，需要创建 manager bean，同时配置 tx:advice 标签，然后手动启用 aop:config 将配置标签和切点关联起来即可。Spring 注解提供了对于事务的传播行为、超时、只读属性、回滚策略、隔离级别进行快速设置的方法，是用注解还是 XML 管理事务取决于项目大小和项目属性。</p></span>

# JDBCTemplate

JDBCTemplate 是一个类似于 DBUtils 的 JDBC 支持工具，而并非 ORM 框架。在 Spring 中，直接使用 JDBCTemplate 来编写 DAO，而不是使用 DBUtils 更加的方便，这两者使用方法类似，但是也有差别。

首先，在 Spring 中使用数据库一般需要通过 XML 手动声明 bean 或者注解自动扫描的方式进行。不论哪一种情况，都需要创建 DataSource 数据源，使用 JDBCTemplate 数据库操纵工具来管理数据源。一个通过 XML 手动声明 bean 的例子如下：

```xml
<bean id="dataSource" class="com.mchange.v2.c3p0.ComboPooledDataSource">
    <property name="user" value="corkine" />
    <property name="password" value="password" />
    <property name="jdbcUrl" value="jdbc:mysql:///log" />
    <property name="driverClass" value="com.mysql.jdbc.Driver" />
    <property name="initialPoolSize" value="5" />
    <property name="maxPoolSize" value="10" />
</bean>

<bean id="temp" class="org.springframework.jdbc.core.JdbcTemplate">
    <property name="dataSource" ref="dataSource"/>
</bean>
```

之后初始化 context，加载 bean 即可：

```java
ApplicationContext context = new ClassPathXmlApplicationContext("spring-temp.xml");
JDBCTemplate template = (JdbcTemplate) context.getBean("temp");
```

## Update

因为 JDBCTemplate 集成了 DataSource，因此不用像 DBUtils 那样传入连接才能进行 update。使用 JDBCTemplate 进行更新（没有返回值的任何数据库操作，包括更新、删除和插入）如下：

```java
String sql = "update customers set name = ? where id = ?";
template.update(sql,"Marvin",10522);
```

批量更新采用 `batchUpdate` 方法，这里需要注意，传入参数是一个数组列表，其中列表对应批量的语句，数组对应每条语句的占位符。

```java
String sql = "update customers set name = ? where id = ?";
List<Object[]> list = new ArrayList<>();
list.add(new Object[]{"Lili",10523});
list.add(new Object[]{"Lili2",10524});
list.add(new Object[]{"Lili3",10525});
template.batchUpdate(sql,list);
```

## Query

**使用 queryForObject(sql, rowMapper) 查询一个对象**

查询并且返回对象，不需要使用 BeanUtils 等工具手动构建，而是需要指定对象的类，采用 `queryForObject` 方法进行对象查询并返回，注意，此方法只返回对象，并且只返回一个对象。

```java
String sql = "select * from customers where id = ?";
RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
Customer customer = template.queryForObject(sql,mapper,521);
System.out.println(customer);
```

注意查询需要传入一个 RowMapper，这个 RowMapper 的泛型用于强行转换最后返回的类型，在这里是 Customer 类型，这个 RowMapper 接受的构造器参数类型将用来作为实际装载数据库信息的 beans。这样的分配看起来很奇怪，但是如果你想要使用一种类型来装载 beans，而获取另一种引用，这种方法就很不错了。

此外，如果想要获取一组对象的话，这里的 RowMapper 不需要改变，但是方法变了！！！！使用 query 方法查询多个对象。

**使用 queryForObject(sql, Class<T\>) 查询一个值**

此外还需要区分 `queryForObject（sql，Class<T>）` 方法，此方法用来将查询的值转换为参数中的对象类型（即用来强制转换的，而不是作为 bean 构造的），此方法用于获取单个数据库值，并转换成指定类型。

```java
String sql = "select count(id) from customers";
Long res = template.queryForObject(sql,Long.class);
System.out.println(res);
```

**使用 query(sql, rowMapper) 查询多个对象**

如果需要查询并返回一个列表的对象，使用 query 方法：

```java
String sql = "select * from customers where id < ?";
RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
List<Customer> list = template.query(sql,mapper,1000);
System.out.println(list);
```

注意，在这种情况下，我们依然使用的是 `BeanPropertyRowMapper` 方法，在 JDBCTempalte 中的 mapper 没有返回列表、Map等不同结构的包装器，这和 DBUtils 对于列表对象返回采用不同的 ResultRowHandler 不同。在 JDBCTemplate 中，使用 query 方法而不是 queryForObject 方法来返回列表对象，以此作区分。

注意，查询多个对象不要使用 `queryForList` 方法。

**总结**

总而言之，rowMapper 没有多种包装器，这是 JDBCTemplate 和 DBUtils 最大的不同。当查询单个值/对象时，使用 queryForObject，此方法传入类型参数可返回一个值的类型，传入 rowMapper 可构造为指定类型 bean 并转型为指定泛型类型。当查询多个对象时，使用 query 方法，传入 rowMapper 即可。

# NamedParameterJdbcTemplate

具名的 JDBCTemaple 在构造 bean 时需要向构造器传入参数，并且其没有无参构造器，这是和普通的 JdbcTemplate 不同的地方(普通版本是通过 property 方式传入数据源的)：

```xml
<bean id="nTemp" class="org.springframework.jdbc.core.namedparam.NamedParameterJdbcTemplate">
    <constructor-arg ref="dataSource" />
</bean>
```

```java
ntemplate = (NamedParameterJdbcTemplate) context.getBean("nTemp");
```

## Query and Update

具名的模板工具更新起来的优点在于，可以使用 `:abc` 来替代问号的占位符，这样的好处在于，如果一个 SQL 语句有很多占位符，那么采用引用的方式更加清晰一些。

```java
String sql = "insert into customers (id,name,email,birth) values (:id,:name,:email,:birth)";
Map<String,Object> map = new HashMap<>();
map.put("id",33);
map.put("name","Corkine Ma");
map.put("email","cm@muninn.cc");
map.put("birth",new Date(new java.util.Date().getTime()));
ntemplate.update(sql,map);
```

更新时需要传入一个 Map，此 Map 的 key 为引用名称， value 为对应需要填充的值。

此外，我们可以传入一个对象，从这个对象身上拔取属性自动填入占位符：

```java
String sql = "insert into customers (id,name,email,birth) values (:id,:name,:email,:birth)";
Customer obj = new Customer(40,"Lili","li@muninn.cn",new Date(new java.util.Date().getTime()));
SqlParameterSource psource = new BeanPropertySqlParameterSource(obj);
ntemplate.update(sql,psource);
```

在这种情况下，我们需要传入 `SqlParameterSource` 这个对象进行更新，而这个对象则通过 `BeanPropertySqlParameterSource` 这个实现类来接受一个 bean，然后从这个 bean 上自动拔取属性填入占位符要求的属性字段。

# JdbcDaoSupport and DAO

JdbcDaoSupport 希望我们在自己编写 DAO 的时候继承 JdbcDaoSupport 类，这个类提供了 getDataSource 的方法，封装很简单。但是继承这个类依旧需要在构造器传入 DataSource，因此颇为鸡肋，一般不使用。

一个基本的 DAO，在 Spring 中如下形式：

```java
@Repository
public class CustomerDao {
    private JdbcTemplate template;

    @Autowired
    public CustomerDao(JdbcTemplate template) {
        this.template = template;
    }

    public Customer get(Integer id) {
        String sql = "select * from customers where id = ?";
        RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
        Customer customer = template.queryForObject(sql,mapper,id);
        return customer;
    }
}
```

因为 Spring 官方不建议直接装配私有方法（装配私有方法造成 Spring 的侵入性太强），因此这里我们使用了构造器来进行装配（或者可以使用 getter/setter）。

注意，开启 DAO 的注解自动装配，需要启用注解自动扫描，对于 XML 手动配置而言，需要声明：

```xml
<context:component-scan base-package="com.mazhangjing.spring.jdbc" resource-pattern="*.class" >
</context:component-scan>
```

# Spring  Transaction

Spring 对于事务提供支持。因为事务的代码固定（在建立连接后关闭自动提交，在全部 statement 后总的提交），因此 Spring 提供了一个叫做 TransactionManager 的类，只要实例化此类，并且指定事务需要的切入点即可完成事务。

## 基于 @Transcational 的注解

最简单的方式是基于注解的，只要说明哪个方法需要启用事务，对其添加 `@Transcational` 即可。注意，这个注解有 sql 包和 spring 包两种实现，后者提供了再注解上直接配置事务传播、回滚、超时等等的方法，但是前者没有。

对于基于注解的方式，需要手动启用事务管理器，启用事务的注解管理方法，然后在业务层添加切点注解即可：

```xml
<bean id="transactionManager" class="org.springframework.jdbc.datasource.DataSourceTransactionManager">
        <property name="dataSource" ref="dataSource"/>
</bean>
<tx:annotation-driven transaction-manager="transactionManager" />
```

首先是持久层。我们在这里创建了一个 DAO 接口，一个 DAO 的接口的实现（并将其添加到 Spring Bean 中）：

```java
public interface BookShopDao {
    int findBookPriceByIsbn(String isbn);
    void updateBookStock(String isbn);
    void updateUserAccount(String username, int price);
}
@Repository("bookShopDao")
public class BookShopImpl implements BookShopDao {

    private JdbcTemplate template;

    @Autowired public BookShopImpl(JdbcTemplate template) { this.template = template; }

    @Override
    public int findBookPriceByIsbn(String isbn) {
        String sql = "select price from book where isbn = ?";
        return template.queryForObject(sql,Integer.class,isbn);
    }

    @Override
    public void updateBookStock(String isbn) {
        //书的库存必须足够
        String check = "select stock from book_stock where isbn = ?";
        int stock = template.queryForObject(check,int.class,isbn);
        if (stock == 0) {
            throw new BookStockException("库存不足");
        }
        String sql = "update book_stock set stock = stock - 1 where isbn = ?";
        template.update(sql,isbn);
    }

    @Override
    public void updateUserAccount(String username, int price) {
        //余额必须足够
        if (template.queryForObject(
                "select balance from account where username = ?", int.class,username) < price)
            throw new UserAccountException("余额不足");
        String sql = "update account set balance = balance - ? where username = ?";
        template.update(sql,price,username);
    }
}
```

之后是业务层，我们创建了一个服务的接口，以及一个服务的实现，服务的实现添加到 bean 中。其中启用了自动注解。

```java
public interface BookShopService {
    void purchase(String username, String isbn);
}
@Service("bookShopService")
public class BookShopServiceImpl implements BookShopService {

    @Autowired public void setBookShopDao(BookShopDao bookShopDao) {
        this.bookShopDao = bookShopDao;
    }

    private BookShopDao bookShopDao;

    @Transactional(propagation = Propagation.REQUIRES_NEW, isolation = Isolation.READ_COMMITTED,
    noRollbackFor = {NoSuchException.class}, readOnly = false, timeout = 5)
    public void purchase(String username, String isbn) {
        try {
            TimeUnit.SECONDS.sleep(1);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        int price = bookShopDao.findBookPriceByIsbn(isbn);
        bookShopDao.updateBookStock(isbn);
        bookShopDao.updateUserAccount(username,price);
    }
}
```

注意这里的注解，在方法上添加即可。我们的 xml 配置如下：

```xml
<context:component-scan base-package="com.mazhangjing.spring">
</context:component-scan>

<bean id="transactionManager" class="org.springframework.jdbc.datasource.DataSourceTransactionManager">
    <property name="dataSource" ref="dataSource"/>
</bean>

<tx:annotation-driven transaction-manager="transactionManager" />
```

首先是自动扫描注解，添加 bean。其次是 transactionManager 管理器，这个类没办法添加注解，因此需要手动在 xml 声明。之后，通过声明事务的注解驱动来启用事务注解的扫描，程序会自动在 @Transcational 注解的位置注入事务代码，完成事务。

总结：在 Spring 中启用事务需要有： DataSource bean、JDBCTemplate bean、BeanDAO bean、BeanService bean、TransactionManager bean。为了获取这些 bean，可以使用注解+注解自动扫描，也可以完全手动 xml 配置。其中数据源是为 template 服务的，而 template 是为 dao 服务的，而 dao 是为 service 服务的。在 dao 之前，都属于持久层， 应该使用 @Repository 注解。在 dao 之后，都属于业务层，应该使用 @Service 注解。

在业务层最底层就是 beanService，之后是事务，事务作用于业务层，manager bean 中的方法被注入 beanService 指定注解的地方。

**因此我们所需要的：创建 bean（不论手动还是自动）、添加 transactionManager、启用注解事务命令，将 manager 注入服务。**


## 事务的传播行为、隔离级别、回滚、只读和超时

在业务层，我们新添加一个接口以及其实现，添加到 Spring bean 中，相关依赖进行注入，启用事务注解，注入注解管理器代码。

```java
public interface Cashier {
    void checkOut(String username, List<String> isbns);
}
@Service("cashier")
public class CashierImpl implements Cashier {

    private BookShopService bookShopService;

    @Autowired public void setBookShopService(BookShopService bookShopService) {
        this.bookShopService = bookShopService;
    }

    @Transactional(propagation = Propagation.REQUIRED)
    @Override
    public void checkOut(String username, List<String> isbns) {
        for (String isbn : isbns) {
            bookShopService.purchase(username,isbn);
        }
    }
}
```

事务具有一定的属性，比如超时、回滚级别、读写级别、传播行为，Spring 注解事务可以直接在注解内传入参数进行设置。

```java
@Transactional(propagation = Propagation.REQUIRES_NEW, isolation = Isolation.READ_COMMITTED,
noRollbackFor = {NoSuchException.class}, readOnly = false, timeout = 5)
```

- 使用 propagation 指定事务传播行为，此行为只针对此方法加入其他事务时起作用
- 使用 isolation 指定隔离级别，最常用为读已提交。
- 使用 no/rollbackFor 指定回滚，默认情况下 spring 的声明式事务对所有运行时异常进行回滚。设定后就不是事务了
- 使用 readOnly 指定事务是否为只读，如果只是读取信息，那么引擎可以对查询进行优化（不加锁）。
- 使用 timeout 指定强制回滚之前，事务可以占用的时间。

**传播行为**

其中传播行为指的是如果两个方法，a和b，其中b在a中执行，a和b都是事务，那么在执行b的时候，应该用b的事务还是a的事务呢？默认情况下，这时候会自动合并事务，合并成a的事务。但是，你可以使用 `Propagation.REQUIRES_NEW` 来指定 b 的特殊传播行为。当在 a 中执行 b 的时候，会检查 b 的事务传播属性，如果是 NEW，则挂起 a，执行b，之后恢复 a，否则按照默认，直接合并为 a 执行。

注意 cashier 和 bookshopService 都设置了事务，因为 service 在内循环，因此 Spring 会扫描到 service 的注解是请求新的事务，那么就会将 cashier 挂起，执行 service。而 cashier 的事务传播注解，只有当其被作为内循环的时候才会生效执行。（换句话说，传播行为的设置是针对被包含的情况的，而不是针对包含别人的情况的/在出现包含被包含关系时使用被包含的注解的设置。）

**隔离级别**

其中隔离默认是读已提交的数据，可能存在两次读结果不同的现象。所有更改都针对注解的方法的这个事务执行的情况，而对于别的事务没有影响。如果事务被合并，则这里设置的属性失效，按照外部事务属性来操作。

**回滚设置**

其中回滚默认即可，这里可以设置对于某些错误不进行回滚，当出现这些错误的时候，不对从此点到出错位置对 SQL 修改的数据进行恢复。但是这样就不再是事务了。

**只读属性**

其中只读非常适用于只读的事务，可以不加锁，优化访问性能。

**超时设置**

其中超时限定事务最长可以执行的时间，如果超过这个时间，则直接回滚，不进行操作，并且报超时错误。

## 基于 XML 的手动配置方式

上面基于注解的方式其实是以 XML 作为总的配置来获取 ApplicationContext 的。其实也不能算纯粹的"基于注解"，只能算作半自动（自动装配 bean、自动事务注入）。而在本小节重点介绍的将是完全手动的基于 XML 的配置。

其中大致流程类似，首先初始化 dataSource bean、JdbcTemplate bean、DAO bean、Service bean、transatcionManager bean。如下：

```xml
<bean id="dataSource" class="com.mchange.v2.c3p0.ComboPooledDataSource">
    <property name="user" value="corkine" />
    <property name="password" value="password" />
    <property name="jdbcUrl" value="jdbc:mysql:///log?useSSL=false" />
    <property name="driverClass" value="com.mysql.jdbc.Driver" />
    <property name="initialPoolSize" value="5" />
    <property name="maxPoolSize" value="10" />
</bean>

<bean id="temp" class="org.springframework.jdbc.core.JdbcTemplate">
    <property name="dataSource" ref="dataSource"/>
</bean>
<bean id="bookShopDao" class="com.mazhangjing.spring.trans.BookShopImpl" >
    <constructor-arg name="template" ref="temp" />
</bean>

<bean id="bookShopService" class="com.mazhangjing.spring.trans.BookShopServiceImpl">
    <property name="bookShopDao" ref="bookShopDao" />
</bean>

<bean id="cashier" class="com.mazhangjing.spring.trans.CashierImpl" >
    <property name="bookShopService" ref="bookShopService"/>
</bean>

<bean id="transactionManager" class="org.springframework.jdbc.datasource.DataSourceTransactionManager">
    <property name="dataSource" ref="dataSource"/>
</bean>
```

这里所做的其实就是自动 bean 配置的手动版本，接下来本来应该是开启基于注解的事务，这里不使用注解，所以需要手动使用 tx:advice 标签设置对于指定方法 tx:method 的属性。

```xml
<tx:advice id="interceptor" transaction-manager="transactionManager" >
    <tx:attributes>
        <tx:method name="purchase" propagation="REQUIRES_NEW" isolation="READ_COMMITTED"/>
        <tx:method name="get*" read-only="true" />
        <tx:method name="find*" read-only="true" />
        <tx:method name="*"/>
    </tx:attributes>
</tx:advice>
```

这里的内容就相当于事务注解时的参数设置。因为注解有位置信息，而这里没有，所以需要使用 tx:method 方法指定。这里的 method 可以使用通配符，一般而言，对于 get 和 find，我们都使用 read-only 的设置，加快访问，对于没有配置的，则按照默认，使用 * 通配符。对于特殊的方法，采用特殊的设置（放在前面）。

现在我们设置好了针对不同方法工作的不同事务属性，下一步就是将其“切”到相应的位置。注意，这里的 method 并非是定义切点，这个 advice 就相当于一个小的配置文件，这里的 method 只是向 manager 说明在何处应该采取何种配置罢了。

切点还是需要使用 aop 标签配置：

```xml
<aop:config>
    <aop:pointcut id="pointcut" expression="execution(* com.mazhangjing.spring.trans.*.*(..))"/>
    <aop:advisor advice-ref="interceptor" pointcut-ref="pointcut"/>
</aop:config>
```

区别于基本的注解，这里使用的是 aop:advisor 标签，指定切点以及对应的 tx:advice 管理器参数即可。

## 开发者面临的选择

这里的配置，基本上就相当于启用事务注解，并且在注解添加参数，并且自动启用 tx:annotation-driven 的标签的效果了。其实这两者差不太多，完全手动的方式中大部分配置属性和切点的标签我们使用注解的时候也要写，只不过是换了一种形式——参数而不是标签。并且基于注解的方式配置分散在各个业务层Java类中，这也不是很方便管理。

至于最好的解决办法，可能是启用 bean 自动配置，然后使用 tx:advice 和 aop:config 以及 aop:advisor 手动配置切点并且统一管理属性来的方便。

不过如果考虑到面向切面的含义，对于一般切面而言，我们会建立一个 java 切面类进行基于注解的切面设置。而对于事务，考虑到代码量本来也不大，属性较少，直接放在业务层问题也不大，当然，如果业务层庞大复杂，进行统一管理，放置在 XML 文件中，或许也是一个不错的选择。

至于将事务放置在Java类中，使用基于注解的切面管理，除非你打算自己写事务代码，或者手动调用 transactionManager，否则不推荐。

总而言之，因为我们有了一个方便的事务管理器，可选的方案有以下几种：要么基于 xml 完全手工，要么基于 xml 半自动（自动装配、手动事务），要么基于 注解 + xml（启用自动注解扫描+自动事务注解注入），要么基于 注解 + java 类，或者混用 xml，注解，java类。